In [1]:
#importing required libraries
import numpy as np
import keras,os
import cv2
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from astroNN.datasets import galaxy10
from tensorflow.keras import utils

WARNING! APOGEE environment variable SDSS_LOCAL_SAS_MIRROR not set
WARNING! Gaia environment variable GAIA_TOOLS_DATA not set


In [2]:
def preprocess(images):
    """ converting images from RGB to Grayscale and normalize them
    Parameter
    ---------
    images : list of images
        The images to convert to grayscale

    Return
    -------
        Grayscale images
    """
#     gr_images = []
#     for im in images:
#         img_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
#         gray = cv2.cvtColor(img_gray, cv2.IMREAD_GRAYSCALE)
#         gr_images.append(gray)
     
    
#     #converting the list to array
#     img_arr = np.array(gr_images)
    
    #normalizing the images
    images = images/255
    
    return images

In [3]:
#loading the dataset
images, labels = galaxy10.load_data()

C:\Users\Faizan\.astroNN\datasets\Galaxy10.h5 was found!


In [4]:
# To convert the labels to categorical 10 classes
labels = utils.to_categorical(labels, 10)

#converting the arrays to float type
labels = labels.astype(np.float32)
images = images.astype(np.float32)

In [5]:
images = preprocess(images)

In [6]:
images.shape

(21785, 69, 69, 3)

In [7]:
X = images
y = labels

In [8]:
#splitting the data into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [9]:
#printing the shapes of test and train data
print('Size of X_train', X_train.shape)
print('Size of y_train', y_train.shape)
print('Size of X_test', X_test.shape)
print('Size of y_test', y_test.shape)

Size of X_train (19606, 69, 69, 3)
Size of y_train (19606, 10)
Size of X_test (2179, 69, 69, 3)
Size of y_test (2179, 10)


#Building the VGG16 Model

In [10]:
model = Sequential()
#first layer
model.add(Conv2D(input_shape=(69,69,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

#second layer
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

#third layer
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

#fourth layer
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

#fifth layer
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

#dense layer
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=10, activation="softmax"))

opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 69, 69, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 69, 69, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 34, 34, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 17, 17, 256)       2

In [12]:
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, 
                             save_best_only=True, save_weights_only=False, mode='auto', period=1)

early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, 
                      mode='auto')

#Training

In [13]:
hist = model.fit(X_train, y_train, steps_per_epoch=50, validation_steps=10,epochs=5,callbacks=[checkpoint,early])

Epoch 1/5
50/50 [==============================] - 1654s 33s/step - loss: 12.1048 - accuracy: 0.2980
Epoch 2/5
50/50 [==============================] - 1569s 31s/step - loss: 1.7482 - accuracy: 0.3201
Epoch 3/5
50/50 [==============================] - 1562s 31s/step - loss: 1.7508 - accuracy: 0.3074
Epoch 4/5
50/50 [==============================] - 1561s 31s/step - loss: 1.7448 - accuracy: 0.3135
Epoch 5/5
50/50 [==============================] - 1559s 31s/step - loss: 1.7450 - accuracy: 0.3212


In [16]:
model.save('galaxyModel2')

INFO:tensorflow:Assets written to: galaxyModel2\assets


In [20]:
model.predict(X_train[10])

ValueError: in user code:

    C:\Users\Faizan\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:1478 predict_function  *
        return step_function(self, iterator)
    C:\Users\Faizan\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Faizan\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Faizan\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Faizan\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Us